### Correlation filter

#### This is the third step of the enrichment calculation

Some Org-IP samples may have very similar profiles, which may lead to underestimation of significance if outlier proteins overlap. 

To tackle this problem, enrichment values are pre-calculated using every other sample in the batch, and these enrichment values are correlated across samples.

The correlation values are then used to filter out highly correlated org-IP samples from being used in the significance test together.

In [1]:
import os
import sys
from datetime import datetime
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

script_path = Path.cwd().parent.parent / "script"
data_path = Path.cwd().parent.parent / "data"
sys.path.append(str(script_path))
from pyseus import primary_analysis as pa
from pyseus import spatial_tools as st

### Load the imputed IP table
The correct datestamp is required to find the input files

In [2]:
# %store -r timestamp USE_FROZEN
# if USE_FROZEN:
#     raise Exception("USE_FROZEN is true, you probably want to skip enrichment and proceed from Fig1")
timestamp = "2024-07-24"
print(f"Timestamp: {timestamp}")

outprefix = f"{timestamp}_QC_filter_impute"

outdir = Path.cwd() / "output"
preprocessing_out = outdir / f"preprocessing"

Timestamp: 2024-07-24


In [3]:
#### Load bait imputed tables (IPs)
IP_path = preprocessing_out / f"{outprefix}_imputed_table.csv"

try:
    bait_imputed_table = pd.read_csv(IP_path, header=[0, 1], index_col = 0)
except FileNotFoundError:
    print(f"File {IP_path} not found.\nPlease run 1.QC_filter_and_impute.ipynb first or specify the correct timestamp, current value is {timestamp}")
except pd.errors.ParserError:
    print(f"There was an error parsing the CSV file at {IP_path}.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [4]:
# #check the WTs
# for i in bait_imputed_table.columns:
#     if "WT" in i[0]:
#         print(i)

### Correlation filter

setup the batches accordin to `2.Batch_selection.ipynb`

In [5]:
batches = [
    ["12", "13"],
    ["01", "03", "04", "05"],
    ["02", "06", "07"],
    ["14", "15", "17"],
    ["09", "10", "11"],
]

In [6]:
# imputed, preprocessed table from the earlier steps
grouped = bait_imputed_table.copy()

# initiate lists to add dataframes for concatenation at the end 
volcano_tables = []
enrichment_tables = []
volcano_tables_againstWT = []
enrichment_tables_againstWT = []

# create a folder to save the correlation tables
correlation_table_dir = outdir / "correlation_tables"
os.makedirs(correlation_table_dir, exist_ok=True)

# iterate through batches
for i, batch in enumerate(batches):
    print('batch: ' + str(i))
    new_cols = []
    # select all the samples from each batch
    for col in list(grouped):
        if col[0] == 'metadata':
            new_cols.append(col)
        # identifying each batch
        elif col[1].split('-')[0] in batch: # splitting the dash and seeing if that experiment is in the batch you're running
            new_cols.append(col) 
    selected = grouped[new_cols].copy()

    #print(list(selected))
    # initial calculation of complement enrichments to identify highly correlated samples
    analysis = pa.AnalysisTables(grouped_table=selected, auto_group=False)
    analysis.generate_export_bait_matrix()

    #analysis.select_wildtype_controls(wt_re='-WT') # select only WT as controls, this is onfirmed by Manu to be the better than using all samples as the intial controls
    analysis.select_matching_WT_control(wt_re='-WT') # select matching WT as controls.

    #print(analysis.exclusion_matrix) # uncomment to check the exclusion matrix
    analysis.exclusion_matrix.to_csv(correlation_table_dir / f"{timestamp}_negative_control_batch_{i}.csv", index=False)

    
    analysis.simple_pval_enrichment(std_enrich=False)
    analysis.convert_to_enrichment_table()
    complements = analysis.enrichment_table.copy()
    control_mat = analysis.exclusion_matrix.copy()

    enrichment_tables_againstWT.append(analysis.enrichment_table.copy()) # export the enrichments against WT
    volcano_tables_againstWT.append(analysis.simple_pval_table.copy()) # export the pvals against WT
    
    # Using the previous calculations, find highly correlated sample (filter is at >0.4)
    spatial = st.SpatialTables(
        preprocessed_table=selected,
        enrichment_table=complements,
        control_mat=control_mat)
    spatial.enrichment_corr_control_mat(corr=0.35)

    # save the correlation table for each batch
    spatial.corr_mat.to_csv(correlation_table_dir / f"{timestamp}_corr_table_batch_{i}.csv", index=False)  
    spatial.sample_corrs.to_csv(correlation_table_dir / f"{timestamp}_corr_val_table_batch_{i}.csv", index=True)
    
    # calculate the final pval and enrichment table for the batch
    spatial.new_corr_ARI(just_enrichment=True, labels=None, reference=None, std_enrich=False)

    # appending respective dataframes to the concatenation list
    volcano_tables.append(spatial.corr_pval_table)
    enrichment_tables.append(spatial.corr_enrichment_table)
    

batch: 0
P-val calculations..
Finished!
P-val calculations..
Finished!
batch: 1
P-val calculations..
Finished!
P-val calculations..
Finished!
batch: 2
P-val calculations..
Finished!
P-val calculations..
Finished!
batch: 3
P-val calculations..
Finished!
P-val calculations..
Finished!
batch: 4
P-val calculations..
Finished!
P-val calculations..
Finished!


The final step is to concatenate results from the batches. 

The enrichment table can be used for multiple purposes, including UMAP generation.

In [7]:
# concatenating the final enrichment table
enrichment_metadata = enrichment_tables[0].loc[:, ["metadata"]].copy()
enrichment_samples = [x.drop(["metadata"], axis=1, level=0) for x in enrichment_tables]

final_enrichments = pd.concat([enrichment_metadata] + enrichment_samples, axis=1)

# concatenating the final volcano table
volcano_metadata = volcano_tables[0].loc[:, ["metadata"]].copy()
volcano_samples = [x.drop(["metadata"], axis=1, level=0) for x in volcano_tables]

final_volcano = pd.concat([volcano_metadata] + volcano_samples, axis=1)

In [8]:
# saving the final tables to files
enrich_out_dir = outdir / "enrichment_and_volcano_tables"
os.makedirs(enrich_out_dir, exist_ok=True)

volcano_csv_path = enrich_out_dir / f"{timestamp}_volcano_table.csv"
enrichment_csv_path = enrich_out_dir / f"{timestamp}_enrichment_table.csv"

final_volcano.to_csv(volcano_csv_path)
final_enrichments.to_csv(enrichment_csv_path)

In [9]:
# check the enrichment table
final_enrichments

metadata  \
                                            Protein IDs   
0         A0A023T6R1;Q96A72;F5H6P7;F5H6N1;F5H3U9;F5H124   
1                          Q9Y5S9;A0A023T787;A0A0J9YW13   
2     A0A0C4DFM1;A0A024QYR3;Q92544;B4DH88;B4DKC1;Q6Z...   
3     A0A024QYR6;A0A1V0DNR7;A0A6G6A825;F6KD02;F6KD01...   
4                              Q99805;A0A024QYR8;B3KSG9   
...                                                 ...   
8591                                             X5D7P8   
8592                                             X5D8X9   
8593  X5DQV1;X5DNI1;B3KV96;E9PD68;B3KXQ5;Q14194;B3KT...   
8594                                             X5DQZ7   
8595                                             X6RHX1   

                                                                          \
                                   Majority protein IDs       Gene names   
0                       A0A023T6R1;Q96A72;F5H6P7;F5H6N1  FLJ10292;MAGOHB   
1                                     Q9Y5S9;A0A023T787       RBM8A;RBM8   
2     A0A0C4DFM1;A0A024QYR3;Q92544;B4DH88;B4DKC1;Q6ZTK5           TM9SF4   
3     A0A024QYR6;A0A1V0DNR7;A0A6G6A825;F6KD02;F6KD01...             PTEN   
4                              Q99805;A0A024QYR8;B3KSG9           TM9SF2   
...                                                 ...              ...   
8591                                             X5D7P8            STK39   
8592                                             X5D8X9          CNTNAP2   
8593  X5DQV1;X5DNI1;B3KV96;E9PD68;B3KXQ5;Q14194;B3KT...            CRMP1   
8594                                             X5DQZ7             GPX1   
8595                                             X6RHX1            SCYL3   

        sample                                                              \
      12-LAMP1  13-RAB1A     12-WT 12-TOMM20 12-PNPLA2 12-SEC61B   12-ACTB   
0    -0.141427  0.164993 -0.403814 -0.279302 -0.386914  0.018797  0.288365   
1    -0.588500 -1.931700  1.457350 -2.246100  0.710800 -0.623300  0.860750   
2     3.703700  6.304299 -3.994900 -2.356032 -5.046599  4.604981 -4.326681   
3     0.261350  0.521100 -0.889284  0.744454  0.087087  0.297697 -0.356495   
4     5.499848  7.657225 -5.625948 -1.099823 -8.018554  6.447580 -7.542662   
...        ...       ...       ...       ...       ...       ...       ...   
8591 -0.030088 -0.838131 -0.750013 -0.523289 -0.998200  0.547829  0.125618   
8592 -0.052453 -0.518175  0.122136 -0.658738  0.072154  0.028252  0.861863   
8593  0.327787  0.309068 -0.205879  0.558718 -0.161728 -0.120634  0.805817   
8594  0.261512 -0.491176 -0.488960  2.629682 -0.817804 -0.382702 -0.287952   
8595  0.746831 -1.629260 -0.200211 -0.430547  0.021381 -0.954216 -0.148533   

      ...                                                                \
      ... 09-HSP90AA1   10-RTN4 09-TOMM20 11-CEP350 09-ATG101  10-EXOC2   
0     ...   -1.310187 -0.369807 -0.294506  0.212616  3.018858 -1.121111   
1     ...   -2.443700 -1.899050 -2.430950  0.895300 -0.728450  1.017250   
2     ...   -5.186684  1.852200 -2.612500  0.397300  2.393000 -4.497900   
3     ...   -0.195980  0.269211 -0.420635 -0.067346 -0.332397 -0.818591   
4     ...   -3.389100  1.524950 -1.385850 -0.438800 -0.412350  0.033350   
...   ...         ...       ...       ...       ...       ...       ...   
8591  ...   -0.147448  0.818616 -0.715761 -0.227490  0.874124 -0.075496   
8592  ...   -0.780498 -0.554957 -1.183749  1.758580  0.498581  0.379988   
8593  ...    1.010047 -0.362585  1.048421 -0.334151 -0.265872  0.229145   
8594  ...    3.884103  0.371638  2.954144 -0.481100 -0.570653  0.151380   
8595  ...    0.279959 -1.412437 -0.503901 -0.767265 -0.747004  0.199421   

                                              
     10-TOMM20     10-WT     09-WT  10-AP2B1  
0    -1.547331  2.958600  2.270600 -0.353301  
1    -2.969400  1.972000  1.836900 -0.336600  
2    -2.685600  0.006200  0.315600  0.620900  
3     0.223960 -0.236051  0.287790 -0.083505  
4    -2.382900 -0